In [40]:
import json
import os
import pandas as pd
import csv

manifest = "updated.manifest"
LOCAL_S3 = "/data/raw_data/PolarBears/s3_images/"
with open(manifest) as f:
    content = f.readlines()
# you may also want to remove whitespace characters like `\n` at the end of each line
content = [x.strip() for x in content]
json_items = []
for c in content:
    if c == "":
        continue
    obj = json.loads(c)
    json_items.append(obj)

In [52]:
new_rows = []

for i in range(len(json_items)):
    obj = json_items[i]
    src = obj["source-ref"]
    path = "/".join(src.replace("s3://","").split("/")[2:])
    img_local = os.path.join(LOCAL_S3, path)
    filename = ".".join(os.path.basename(img_local).split(".")[:-1])

    if not 'NOAAlabelsdone' in obj:
        continue
    human_labeled = obj['NOAAlabelsdone']
    meta = obj['NOAAlabelsdone-metadata']
    labels = human_labeled['annotations']
    for label in labels:
        cx,cy,w,h = label['left'],label['top'], label['width'], label['height']
        x1 = int(cx - (w/2))
        x2 = int(cx + (w/2))
        y1 = int(cy - (h/2))
        y2 = int(cy + (h/2))
        PB_ID = ""
        if "PB_ID" in label:
            PB_ID = label['PB_ID']
        STATUS = ""
        if "Age_class" in label:
            STATUS += label['Age_class'] + "-"
        if 'bad-res' in meta and meta['bad-res']:
            STATUS += "bad_res-"
        STATUS=STATUS[:-1]
        if len(STATUS) == 0:
            STATUS = "none"
        new_row = {
            'id': '-1',
            'color_image': os.path.basename(img_local),
            'thermal_image': "",
            'hotspot_id': PB_ID,
            'hotspot_type': "Animal",
            'species_id': "Polar Bear",
            'species_confidence': "",
            'fog': "",
            'thermal_x': "",
            'thermal_y': "",
            'color_left': x1,
            'color_top': y1,
            'color_right': x2,
            'color_bottom': y2,
            'updated_left': x1,
            'updated_top': y1,
            'updated_right': x2,
            'updated_bottom': y2,
            'updated': True,
            'status': STATUS}
        new_rows.append(new_row)
new_df = pd.DataFrame(new_rows)
new_df.to_csv("polar_bears.csv",quoting=csv.QUOTE_NONE, index=False)
